In [ ]:
import numpy as np 
import os

In [ ]:
!cmake -D CUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda  "~/custom_layer/"

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
TF_INC: /usr/local/lib/python2.7/dist-packages/tensorflow_core/include
TF_LIB: /usr/local/lib/python2.7/dist-packages/tensorflow_core
TF_COMPILE_FLAGS: -I/usr/local/lib/python2.7/dist-packages/tensorflow_core/include -D_GLIBCXX_USE_CXX11_ABI=0
TF_LINK_FLAGS: -L/usr/local/lib/python2.7/dist-packages/tensorflow_core -l:libtensorflow_framework.so.2
-- Looking for pthread.h
-- Looking for pthread.h - found


In [ ]:
!make -j

[-33%] Building NVCC (Device) object CMakeFiles/lpg.dir/lpg_generated_local_planar_guidance.cu.o
/usr/local/lib/python2.7/dist-packages/tensorflow_core/include/absl/strings/string_view.h(495): warning: expression has no effect

/usr/local/lib/python2.7/dist-packages/tensorflow_core/include/tensorflow/core/platform/env.h(356): warning: overloaded virtual function "tensorflow::Env::RegisterFileSystem" is only partially overridden in class "tensorflow::EnvWrapper"

/usr/local/lib/python2.7/dist-packages/tensorflow_core/include/absl/types/optional.h(425): warning: expression has no effect
          detected during instantiation of "const T &absl::optional<T>::operator*() const & [with T=stream_executor::dnn::AlgorithmDesc]" 
/usr/local/lib/python2.7/dist-packages/tensorflow_core/include/tensorflow/stream_executor/dnn.h(804): here

/usr/local/lib/python2.7/dist-packages/tensorflow_core/include/absl/types/optional.h(425): warning: expression has no effect
          detected during instantiat

In [ ]:
from tensorflow.keras.layers import Conv2D, UpSampling2D, LeakyReLU, Concatenate, BatchNormalization
import tensorflow as tf

import _local_planar_guidance_grad
lpg = tf.load_op_library('~/custom_layer/liblpg.so')
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2



class Encoder(Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.base_model = MobileNetV2(input_shape=(480,640, 3), include_top=False, weights='imagenet')
        print('Base model loaded')

        outputs = [self.base_model.outputs[-1]]
        for name in ['block_1_expand_relu','block_3_expand_relu','block_6_expand_relu','block_13_expand_relu','block_16_expand_relu'] : outputs.append( self.base_model.get_layer(name).output )
        self.encoder = Model(inputs=self.base_model.inputs, outputs=outputs)

    def call(self, x):
        return self.encoder(x)


class ConvBlock(Model):
    def __init__(self, num_out_layers, kernel_size,stride, activation_fn):
        super(ConvBlock, self).__init__()
        self.convA = Conv2D(filters=num_out_layers, kernel_size=kernel_size,strides=stride,padding='valid', activation=activation_fn)
    
    def call(self, x, kernel_size):
        p = np.floor((kernel_size - 1) / 2).astype(np.int32)
        p_x = tf.pad(x, [[0, 0], [p, p], [p, p], [0, 0]])
        return self.convA(p_x)


class UpConv(Model):
    def __init__(self,num_filters, kernel_size, stride, activation_fn):
        super(UpConv, self).__init__()
        #self.up1 = ConvBlock(num_filters, kernel_size, stride, tf.nn.elu)

    def call(self, x, kernel_size):
        
        upsample = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)
        #conv1 = self.up1(upsample, kernel_size)
        #print(conv1)
        return upsample

class BatchNorm(Model):

    def __init__(self,momentum,epsilon):
        super(BatchNorm,self).__init__()
        self.bn=BatchNormalization(momentum=momentum, epsilon=epsilon,scale=True,fused=True, trainable=True)

    def call(self,x):
        return self.bn(x)


class AtrConv(Model):

    def __init__(self,num_out_layers, kernel_size,rate):
        super(AtrConv,self).__init__()
        self.bn1  = BatchNorm(momentum=0.99, epsilon=1.1e-5)
        self.con1 = Conv2D(num_out_layers * 2, 1, 1, 'same')
        self.bn2  = BatchNorm(momentum=0.99, epsilon=1.1e-5)
        self.con2 = Conv2D(num_out_layers, kernel_size=kernel_size,dilation_rate=rate, strides=1, padding='same',activation=None)
    
    def call(self,out,apply_bn_first = True) :

        if apply_bn_first is True:
            out = self.bn1(out)

        out   = tf.nn.relu(out)
        out   = self.con1(out)
        out   = self.bn2(out)
        out   = tf.nn.relu(out)
        out   = self.con2(out)
        
        return out

class R1x1(Model):

    def __init__(self,num_filters):
        super(R1x1,self).__init__()
        self.convA = ConvBlock(1,1,1,tf.nn.sigmoid)
        self.convB = ConvBlock(3,1,1,None)
        self.convC = ConvBlock(num_filters, 1, 1,tf.nn.relu)
    def call(self,net,num_filters,is_final=False):
        while num_filters >= 4:
            if num_filters < 8:
                if is_final:
                    net = self.convA(net,1)
                else:
                    net = self.convB(net, 3)
                    theta = tf.nn.sigmoid(net[:, :, :, 0]) * 3.1415926535 / 6
                    phi = tf.nn.sigmoid(net[:, :, :, 1]) * 3.1415926535 * 2
                    dist = tf.nn.sigmoid(net[:, :, :, 2]) * 10
                    n1 = tf.expand_dims(tf.multiply(tf.math.sin(theta), tf.math.cos(phi)), 3)
                    n2 = tf.expand_dims(tf.multiply(tf.math.sin(theta), tf.math.sin(phi)), 3)
                    n3 = tf.expand_dims(tf.math.cos(theta), 3)
                    n4 = tf.expand_dims(dist, 3)
                    net = tf.concat([n1, n2, n3, n4], axis=3)
                break
            else:
                net = self.convC(net, 1)

            num_filters = num_filters // 2

        return net

   
class Decoder(Model):
    def __init__(self, num_filters):
        super(Decoder, self).__init__()
        self.max_depth = 10
        self.focal = 519

        self.upconA = UpConv(num_filters, 3, 2, tf.nn.elu) #
        self.upconB = UpConv(num_filters //2 , 3, 2, tf.nn.elu) #
        self.upconC = UpConv(num_filters //4 , 3, 2, tf.nn.elu)  #
        self.upconD = UpConv(num_filters //8 , 3, 2, tf.nn.elu)  #
               
        self.conA   = ConvBlock(num_filters,3,1,tf.nn.elu)#
        self.conB   = ConvBlock(num_filters // 2 , 3,1,tf.nn.elu)#
        self.conC   = ConvBlock(num_filters //4 ,3,1,tf.nn.elu)#
        self.conD   = ConvBlock(num_filters // 4 ,3,1,tf.nn.elu)#
        self.conE   = ConvBlock(num_filters // 8 ,3,1,tf.nn.elu)#
        self.conE   = ConvBlock(num_filters // 16 ,3,1,tf.nn.elu)#
               
        self.bnA    = BatchNorm(momentum=0.99, epsilon=1.1e-5)#
        self.bnB    = BatchNorm(momentum=0.99, epsilon=1.1e-5)#
        self.bnC    = BatchNorm(momentum=0.99, epsilon=1.1e-5)#
        self.bnD    = BatchNorm(momentum=0.99, epsilon=1.1e-5)#
        self.bnE    = BatchNorm(momentum=0.99, epsilon=1.1e-5)#
        
        self.atA    = AtrConv(num_filters // 4  ,3, 3 ) #
        self.atB    = AtrConv(num_filters //4 , 3, 6 ) #
        self.atC    = AtrConv(num_filters //4 , 3, 12) #
        self.atD    = AtrConv(num_filters // 4  ,3, 18 ) #
        self.atE    = AtrConv(num_filters // 4  ,3, 24 ) #
        
        self.r11A   = R1x1 (num_filters // 4)#
        self.r11B   = R1x1 (num_filters // 8)#
       

    
    def downsample_nn(self, x, ratio):
        s = tf.shape(x)
        h = tf.cast(s[1] / ratio, tf.int32)
        w = tf.cast(s[2] / ratio, tf.int32)
        return tf.image.resize(x, [h, w])
    
    def call(self, features):
        x,d1,d2,d3,d4,d5= features[0], features[1], features[2], features[3], features[4], features[5]

        num_filters = 640
        max_depth = 10
    
        upconv5 = self.upconA(x,3)  # H/16
        upconv5 = self.bnA(upconv5)
        
        concat5 = tf.concat([upconv5,d4], 3)
        iconv5 = self.conA(concat5, 3)
        num_filters = num_filters // 2

        upconv4 = self.upconB(iconv5, 3)  # H/8
        upconv4 = self.bnB(upconv4)
        
        concat4 = tf.concat([upconv4,d3], 3)
        iconv4 = self.conB(concat4, 3)
        
        iconv4 = self.bnC(iconv4)
        daspp_3 = self.atA(iconv4, apply_bn_first=False)
        concat4_2 = tf.concat([concat4, iconv4], 3)
        
        daspp_6 = self.atB(concat4_2)
        concat4_3 = tf.concat([concat4_2, daspp_6], 3)
        daspp_12 = self.atC(concat4_3)
        concat4_4 = tf.concat([concat4_3, daspp_12], 3)
        
        daspp_18 = self.atD(concat4_4)
        concat4_5 = tf.concat([concat4_4, daspp_18], 3)
        
        daspp_24 = self.atE(concat4_5)
        concat4_daspp = tf.concat([iconv4, daspp_3, daspp_6, daspp_12, daspp_18, daspp_24], 3)
        
        daspp_feat = self.conC(concat4_daspp,3)
        
        plane_eq_8x8 = self.r11A(daspp_feat, num_filters // 2)
        plane_normal_8x8 = tf.nn.l2_normalize(plane_eq_8x8[:, :, :, 0:3], axis=3)
        plane_dist_8x8 = plane_eq_8x8[:, :, :, 3]
        plane_eq_8x8 = tf.concat([plane_normal_8x8, tf.expand_dims(plane_dist_8x8, 3)], 3)
        depth_8x8 = lpg.local_planar_guidance(plane_eq_8x8, upratio=8, focal=self.focal)
        depth_8x8_scaled = tf.expand_dims(depth_8x8, 3) / 10
        depth_8x8_scaled_ds = self.downsample_nn(depth_8x8_scaled, 4)
        
        num_filters = num_filters // 2
        
        upconv3 = self.upconC(daspp_feat ,3)  # H/4
        upconv3 = self.bnD(upconv3)
        concat3 = tf.concat([upconv3,d2, depth_8x8_scaled_ds], 3)
        #concat3 = tf.concat([upconv3,d2], 3)
        iconv3 = self.conD(concat3, 3)
        
        plane_eq_4x4 = self.r11B(iconv3, num_filters // 2)
        plane_normal_4x4 = tf.nn.l2_normalize(plane_eq_4x4[:, :, :, 0:3], axis=3)
        plane_dist_4x4 = plane_eq_4x4[:, :, :, 3]
        plane_eq_4x4 = tf.concat([plane_normal_4x4, tf.expand_dims(plane_dist_4x4, 3)], 3)
        depth_4x4 = lpg.local_planar_guidance(plane_eq_4x4, upratio=4, focal=self.focal)
        depth_4x4_scaled = tf.expand_dims(depth_4x4, 3) / 10
        depth_4x4_scaled_ds = self.downsample_nn(depth_4x4_scaled, 2)

        num_filters = num_filters // 2    #1/8

        upconv2 = self.upconD(iconv3,  3)  # H/2
        upconv2 = self.bnE(upconv2)
        concat2 = tf.concat([upconv2, d1 , depth_4x4_scaled_ds], 3)
        iconv2 = self.conE(concat2, 3)
        
        plane_eq_2x2 = self.reduction_1x1(iconv2, num_filters / 2)
        plane_normal_2x2 = tf.nn.l2_normalize(plane_eq_2x2[:, :, :, 0:3], axis=3)
        plane_dist_2x2 = plane_eq_2x2[:, :, :, 3]
        plane_eq_2x2 = tf.concat([plane_normal_2x2, tf.expand_dims(plane_dist_2x2, 3)], 3)
        depth_2x2 = lpg.local_planar_guidance(plane_eq_2x2, upratio=2, focal=self.focal)
        depth_2x2_scaled = tf.expand_dims(depth_2x2, 3) / self.max_depth

        num_filters = num_filters / 2

        upconv1 = self.upconD(iconv2, 3)  # H
        reduc1x1 = self.reduction_1x1(upconv1, num_filters, is_final=True)
        concat1 = tf.concat([upconv1, reduc1x1, depth_2x2_scaled, depth_4x4_scaled, depth_8x8_scaled], 3)
        iconv1 = self.conF(concat1, 3)
        
        
        return iconv1

class DepthEstimate(Model):
    def __init__(self):
        super(DepthEstimate, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder( num_filters = int(self.encoder.layers[-1].output[0].shape[-1] // 2 ) )
        print('\nModel created.')

    def call(self, x):
        p=self.encoder(x)
        return self.decoder(p)

In [ ]:
model = DepthEstimate()

9412608/9406464 [==============================] - 0s 0us/step
Base model loaded

Model created.


In [ ]:
batch_size     = 1
learning_rate  = 0.0001
epochs         = 6

In [ ]:
import tensorflow as tf

from io import BytesIO
from zipfile import ZipFile
from sklearn.utils import shuffle

class DataLoader():
    def __init__(self, csv_file='data/nyu2_train.csv', DEBUG=False):
        self.shape_rgb = (480,640, 3)
        self.shape_depth = (480,640, 1)
        self.read_nyu_data(csv_file, DEBUG=DEBUG)
        
    def read_nyu_data(self, csv_file, DEBUG=False):
        csv = open(csv_file, 'r').read()
        nyu2_train = list((row.split(',') for row in (csv).split('\n') if len(row) > 0))
        #nyu2_train=nyu2_train[0:100]
        # Dataset shuffling happens here
        nyu2_train = shuffle(nyu2_train, random_state=0)
        nyu2_train=nyu2_train[1:10]
        # Test on a smaller dataset
        if DEBUG: nyu2_train = nyu2_train[:10]
        
        # A vector of RGB filenames.
        self.filenames = [i[0] for i in nyu2_train]

        # A vector of depth filenames.
        self.labels = [i[1] for i in nyu2_train]

        # Length of dataset
        self.length = len(self.filenames)

    def _parse_function(self, filename, label): 
        # Read images from disk
        #image_decoded = tf.image.decode_jpeg(tf.io.read_file(filename))
        depth_resized = tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(label)), [self.shape_depth[0], self.shape_depth[1]])
        image_decoded = tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(filename)), [self.shape_rgb[0], self.shape_rgb[1]])
        # Format
        rgb = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
        depth = tf.image.convert_image_dtype(depth_resized / 255.0, dtype=tf.float32)
        
        # Normalize the depth values (in cm)
        depth = 1000 / tf.clip_by_value(depth * 1000, 10, 1000)

        return rgb, depth

    def get_batched_dataset(self, batch_size):
        self.dataset = tf.data.Dataset.from_tensor_slices((self.filenames, self.labels))
        self.dataset = self.dataset.shuffle(buffer_size=len(self.filenames), reshuffle_each_iteration=True)
        self.dataset = self.dataset.repeat()
        self.dataset = self.dataset.map(map_func=self._parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        #self.dataset = self.dataset.map(map_func=self.nyu_resize)
        self.dataset = self.dataset.batch(batch_size=batch_size)

        return self.dataset

In [ ]:
import tensorflow
#from loss import depth_loss_function


optimizer = tensorflow.keras.optimizers.Adam(lr=learning_rate, amsgrad=True)

model.compile(loss=depth_loss_function, optimizer=optimizer)

In [ ]:
import os
import tensorflow
checkpoint_path = "/content/drive/My Drive/workspace/btscp/cp.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
import time
a=time.time()
history=model.fit(train_generator, epochs=1, steps_per_epoch=dl.length//batch_size, callbacks=[cp_callback],initial_epoch=0)

In [ ]:
dl = DataLoader()
train_generator = dl.get_batched_dataset(batch_size)

print('Data loader ready.')

Data loader ready.


In [ ]:

import tensorflow.keras.backend as K
'''
def depth_loss_function(y_true, y_pred, theta=0.3, maxDepthVal=1000.0/10.0):
    
    # Point-wise depth
    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

    # Edges
    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)
    l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)

    # Structural similarity (SSIM) index
    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, maxDepthVal)) * 0.5, 0, 1)

    # Weights
    w1 = 1.0
    w2 = 1.0
    w3 = theta

    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))
'''

'\ndef depth_loss_function(y_true, y_pred, theta=0.3, maxDepthVal=1000.0/10.0):\n    \n    # Point-wise depth\n    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)\n\n    # Edges\n    dy_true, dx_true = tf.image.image_gradients(y_true)\n    dy_pred, dx_pred = tf.image.image_gradients(y_pred)\n    l_edges = K.mean(K.abs(dy_pred - dy_true) + K.abs(dx_pred - dx_true), axis=-1)\n\n    # Structural similarity (SSIM) index\n    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, maxDepthVal)) * 0.5, 0, 1)\n\n    # Weights\n    w1 = 1.0\n    w2 = 1.0\n    w3 = theta\n\n    return (w1 * l_ssim) + (w2 * K.mean(l_edges)) + (w3 * K.mean(l_depth))\n'

In [ ]:
import tensorflow.keras.backend as K

def depth_loss_function(y_true, y_pred):
    y_pred=tf.dtypes.cast(y_pred, tf.float32)
    y_true=tf.dtypes.cast(y_true, tf.float32)
    eps=0.0001
    thresh=y_true>eps
    y_tr = y_true[thresh] 
    y_pr = y_pred[thresh] 
    
    logloss = K.mean( K.abs(K.log(y_tr) - K.log(y_pr)), axis=-1)
    return  logloss

In [ ]:
cd /kaggle/input/test-data

In [ ]:
import numpy as np
from io import BytesIO
rgb = np.load('eigen_test_rgb.npy')
depth = np.load('eigen_test_depth.npy')
crop = np.load('eigen_test_crop.npy')
print('Test data loaded.\n')
print(np.shape(rgb))
print(np.shape(depth))

In [ ]:
'''
from skimage.transform import resize
import time
rgb1=[]
depth1=[]
for i in range(len(rgb)):
    rgb1.append(resize(rgb[i], (224,224), preserve_range=True, mode='reflect', anti_aliasing=True ))
for j in range(len(depth)):
    depth1.append(resize(depth[j], (224,224), preserve_range=True, mode='reflect', anti_aliasing=True ))
start = time.time()
print(np.shape(rgb1))
print(np.shape(depth1))
print('Testing...')
rgb1=np.array(rgb1)
depth1=np.array(depth1)
'''

In [ ]:
cd /kaggle/input/evaluateutils


In [ ]:
from utils import predict, load_images, display_images, evaluate
class ev_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    b=time.time()
    e = evaluate(model, rgb, depth, crop, batch_size=1)
    print("eval results ")
    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))
    print("eval time...")
    print(time.time()-b)

In [ ]:
#model.load_weights("/kaggle/input/logloss4/cp.ckpt")

In [ ]:
cd /kaggle/input/extract